In [1]:
import os, sys
# import openai
from openai import AzureOpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import json
from openai import OpenAI

In [2]:
def APIKeyManager(model_type, key_path):
    
    load_dotenv(dotenv_path=key_path, override=True)
    if model_type=='azure':
        client = AzureOpenAI(
            api_version=os.environ["AZURE_API_VERSION"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_API_KEY"],
        )
        return client
    elif model_type=='fanar':
        client = OpenAI(base_url="https://api.fanar.qa/v1",api_key=os.environ["FANAR_API_KEY"],)
    elif model_type=='gemini':
        pass
    return client

# Load environment variables
model_type="azure"
deployment = APIKeyManager(model_type, "./azure.env")


In [3]:
class ImprovedPodcastOutlineGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def generate_outline(self, topic, information, host_persona, guest_persona, style="حواري", duration="30 دقيقة"):
        
        # Style-specific guidance
        style_prompts = {
            "حواري": """
أسلوب حواري:
- ركز على الحوار الطبيعي والتفاعل الشخصي بين المقدم والضيف
- أضف لحظات تداخل وتفاعل عفوي
- استخدم لغة ودية ومألوفة
- اجعل النقاش يبدو كمحادثة بين أصدقاء
- أكثر من الأسئلة الشخصية والتجارب الذاتية""",
            
            "تعليمي": """
أسلوب تعليمي:
- اهتم بتقديم المعلومات بطريقة منظمة ومفصلة
- استخدم أمثلة توضيحية وتشبيهات مفهومة
- اجعل الضيف يشرح المفاهيم خطوة بخطوة
- أضف أسئلة استيضاحية من المقدم
- ركز على الفهم العميق للموضوع مع الحفاظ على الطبيعية""",
            
            "ترفيهي": """
أسلوب ترفيهي:
- أضف عناصر مرحة وقصص شخصية طريفة
- استخدم الفكاهة المناسبة والتعليقات الخفيفة
- اجعل النقاش حيوياً ومليئاً بالطاقة
- أضف مواقف مضحكة أو غريبة مرتبطة بالموضوع
- ركز على الجانب الإنساني والممتع من الموضوع""",
            
            "تحليلي": """
أسلوب تحليلي:
- ركز على التحليل العميق والنقاش المتخصص
- استخدم بيانات وإحصائيات ومراجع علمية
- اطرح أسئلة تحليلية معقدة
- ناقش التحديات والحلول بتفصيل
- اجعل النقاش فكرياً ومتعمقاً مع الحفاظ على الوضوح"""
        }
        
        style_guidance = style_prompts.get(style, "حافظ على التوازن بين المحتوى والتفاعل")
        
        # Duration-specific guidance
        duration_guidance = ""
        if "45" in duration or "طويلة" in duration:
            duration_guidance = "نظراً لطول الحلقة، أضف المزيد من التفاصيل والأمثلة والنقاط الفرعية في كل قسم."
        elif "15" in duration or "قصيرة" in duration:
            duration_guidance = "نظراً لقصر الحلقة، ركز على النقاط الأساسية واجعل المحتوى مكثفاً ومباشراً."
        else:
            duration_guidance = "اجعل المحتوى متوازناً ومناسباً لحلقة متوسطة الطول."

        # Generic cultural integration
        cultural_integration = f"""
السياق الثقافي العربي المرتبط بأي موضوع:

ملاحظة مهمة: الأمثلة التالية هي للإرشاد فقط. يجب تطبيق نفس المبادئ على الموضوع الفعلي '{topic}' وليس استخدام الأمثلة حرفياً.

1. أمثال وحكم مرتبطة بالموضوع:
   - ابحث عن أمثال عربية ترتبط مباشرة بطبيعة الموضوع المطروح
   - مقولات مأثورة تتعلق بالمجال (مثل: العلم، التطور، الحكمة، الصبر، العمل)
   - حكم تتناسب مع روح الموضوع سواء كان تقني، ديني، سياسي، أو اجتماعي

2. مراجع إقليمية ومعاصرة:
   - تجارب المنطقة العربية أو الدول العربية في هذا المجال
   - مبادرات محلية أو خليجية أو عربية ذات صلة
   - شخصيات أو مؤسسات عربية رائدة في المجال
   - أحداث تاريخية أو معاصرة مرتبطة بالموضوع

3. التجارب المشتركة:
   - كيف يتفاعل المجتمع العربي مع هذا النوع من المواضيع
   - تحديات مشتركة نواجهها في المنطقة العربية
   - فرص مستقبلية للعالم العربي في هذا المجال
   - نقاط اهتمام مشتركة للجمهور العربي

4. الربط بالقيم العربية والإسلامية:
   - كيف يتماشى الموضوع مع القيم العربية والإسلامية (إن أمكن)
   - الفوائد المجتمعية من منظور ثقافي عربي
   - التحديات الأخلاقية أو الثقافية من منظور عربي
   - الموازنة بين التطور والحفاظ على الهوية
"""

        # Universal emotional arc
        emotional_arc = f"""
القوس العاطفي للحلقة (قابل للتطبيق على أي موضوع):

ملاحظة: هذا إطار عام يجب تطبيقه على الموضوع الفعلي وليس استخدام الأمثلة حرفياً.

1. البداية (حماس وترقب):
   - المقدم يبدأ بحماس طبيعي حسب شخصيته وعلاقته بالموضوع
   - تشويق للموضوع يناسب خلفية المستمعين العرب
   - ترقب لما سيشاركه الضيف من خبرات

2. الوسط (فضول وتعلم):
   - فضول متزايد من المقدم حول جوانب الموضوع
   - لحظات دهشة عند اكتشاف معلومات جديدة أو غير متوقعة
   - تفاعل إيجابي مع شروحات وتحليلات الضيف

3. التحدي (قلق أو تساؤل):
   - طرح تحديات أو مخاوف مشروعة حول الموضوع
   - نقاش حول الصعوبات أو المقاومة أو الجدل المحيط بالموضوع
   - تساؤلات أخلاقية أو ثقافية أو عملية حسب طبيعة الموضوع

4. الحل والفهم:
   - وصول لفهم أعمق أو منظور جديد مع الضيف
   - إيجاد حلول أو إجابات للتساؤلات المطروحة
   - تفاؤل حذر أو واقعي حسب طبيعة الموضوع

5. النهاية (إلهام ودعوة للعمل):
   - رسالة ملهمة تناسب الجمهور العربي وطبيعة الموضوع
   - خطوات عملية يمكن للمستمعين اتخاذها (إن أمكن)
   - تطلع للمستقبل والدور العربي في هذا المجال
"""

        prompt = f"""
أنت خبير بودكاست عربي متخصص في خلق محتوى تفاعلي أصيل وطبيعي لأي موضوع.

تحذير مهم: جميع الأمثلة والإرشادات المذكورة أدناه هي للتوجيه العام فقط. يجب تطبيق هذه المبادئ على الموضوع الفعلي المطلوب وهو '{topic}' وليس استخدام أي أمثلة حرفياً.

معلومات الحلقة:
الموضوع الفعلي: {topic}
المدة: {duration}
الأسلوب: {style}

{style_guidance}
{duration_guidance}

الشخصيات:
المقدم: {host_persona}
الضيف: {guest_persona}

متطلبات الحوار:
- اجعل كل شخصية تتحدث بأسلوبها المميز حسب خلفيتها
- أضف تفاعلات طبيعية ولحظات تداخل عفوي
- أدرج نقاط اختلاف صحي في وجهات النظر لإضافة أصالة
- اجعل المقدم يتعلم ويتفاعل مع معلومات الضيف الجديدة

{cultural_integration}

{emotional_arc}

المعلومات الأساسية للموضوع:
{information}

تذكر: كل المحتوى يجب أن يكون مرتبطاً بالموضوع الفعلي '{topic}' وليس أي موضوع آخر.

أنشئ مخططاً بصيغة JSON يتضمن العناصر التالية:

{{
    "episode_topic": "الموضوع الرئيسي للحلقة",
    "personas": {{
        "host": {{
            "name": "اسم المقدم",
            "background": "خلفية المقدم",
            "speaking_style": "أسلوب التحدث والنبرة"
        }},
        "guest": {{
            "name": "اسم الضيف", 
            "background": "خلفية الضيف",
            "speaking_style": "أسلوب التحدث والنبرة"
        }}
    }},
    "conversation_flow": {{
        "intro1": {{
            "opening_line": "الجملة الافتتاحية الفعلية للمقدم بالنص المطلوب",
            "podcast_introduction": "تعريف محدد بالبودكاست مرتبط بالموضوع",
            "episode_hook": "جملة تشويقية محددة عن موضوع الحلقة",
            "tone_guidance": "إرشادات النبرة المطلوبة",
            "spontaneity_elements": [
                "عبارات تلقائية محددة يقولها المقدم",
                "تعليقات شخصية مرتبطة بالموضوع"
            ]
        }},
        "intro2": {{
            "topic_introduction": "النص الفعلي لتقديم الموضوع",
            "guest_welcome": "النص الفعلي للترحيب بالضيف",
            "guest_bio_highlight": "تعريف محدد بخلفية الضيف مرتبط بالموضوع",
            "cultural_connections": [
                "ربط ثقافي محدد بالموضوع",
                "مرجع محلي أو إقليمي محدد"
            ],
            "transition_to_discussion": "النص الفعلي للانتقال إلى النقاش الرئيسي"
        }},
        "main_discussion": [
            {{
                "point_title": "عنوان النقطة الأولى",
                "personal_angle": "كيف ترتبط هذه النقطة بخلفية الشخصيات",
                "spontaneous_triggers": [
                    "محفزات تلقائية مرتبطة بهذه النقطة تحديداً",
                    "عبارات طبيعية للحديث عن هذا الجانب من الموضوع"
                ],
                "disagreement_points": "نقاط اختلاف أو جدل محتملة مرتبطة بهذه النقطة تحديداً",
                "cultural_references": [
                    "أمثال ومقولات مرتبطة بهذا الجانب من الموضوع",
                    "مراجع ثقافية محددة ذات صلة مباشرة"
                ],
                "natural_transitions": "عبارات انتقال طبيعية للنقطة التالية مرتبطة بالموضوع",
                "emotional_triggers": "محفزات عاطفية مرتبطة بهذا الجانب من الموضوع"
            }},
            {{
                "point_title": "عنوان النقطة الثانية",
                "personal_angle": "الربط الشخصي بالموضوع",
                "spontaneous_triggers": [
                    "محفزات تلقائية للنقطة الثانية",
                    "عبارات طبيعية مرتبطة بهذا الجانب"
                ],
                "disagreement_points": "نقاط جدل صحية",
                "cultural_references": [
                    "مراجع ثقافية ذات صلة",
                    "تجارب مشتركة"
                ],
                "natural_transitions": "انتقالات سلسة",
                "emotional_triggers": "محفزات عاطفية"
            }},
            {{
                "point_title": "عنوان النقطة الثالثة",
                "personal_angle": "زاوية شخصية للموضوع",
                "spontaneous_triggers": [
                    "محفزات تلقائية للنقطة الثالثة",
                    "عبارات طبيعية لهذا الجانب"
                ],
                "disagreement_points": "نقاط اختلاف بناءة",
                "cultural_references": [
                    "حكم ومأثورات",
                    "قصص تراثية"
                ],
                "natural_transitions": "عبارات ربط طبيعية",
                "emotional_triggers": "لحظات تأثر طبيعية"
            }}
        ],
        "spontaneous_moments": {{
            "natural_interruptions": [
                "نقاط تداخل طبيعية مرتبطة بالموضوع",
                "لحظات مقاطعة إيجابية حول جوانب محددة من الموضوع"
            ],
            "emotional_reactions": [
                "محفزات عاطفية مرتبطة بالموضوع المطروح",
                "لحظات تأثر حقيقية حول جوانب الموضوع"
            ],
            "personal_stories": [
                "قصص شخصية مرتبطة بالموضوع حسب خلفية كل شخصية",
                "تجارب ذاتية متعلقة بالموضوع المحدد"
            ],
            "humorous_moments": [
                "لحظات طريفة مرتبطة بالموضوع",
                "تعليقات مرحة حول جوانب من الموضوع"
            ]
        }},
        "personality_interactions": {{
            "host_strengths": "نقاط قوة المقدم في الحوار",
            "guest_expertise": "مجالات خبرة الضيف",
            "natural_chemistry": "كيف تتفاعل الشخصيتان طبيعياً",
            "tension_points": "نقاط توتر صحية للحوار",
            "collaboration_moments": "لحظات تعاون وتفاهم"
        }},
        "closing": {{
            "conclusion": {{
                "main_takeaways": "النقاط الرئيسية المحددة التي يجب تلخيصها",
                "guest_final_message": "الرسالة الختامية المحددة للضيف",
                "host_closing_thoughts": "أفكار المقدم الختامية المحددة",
                "summary_tone": "نبرة التلخيص المطلوبة"
            }},
            "outro": {{
                "guest_appreciation": "النص الفعلي لشكر الضيف",
                "audience_thanks": "النص الفعلي لشكر المستمعين",
                "call_to_action": "الدعوة المحددة للتفاعل",
                "next_episode_teaser": "التشويق المحدد للحلقة القادمة",
                "final_goodbye": "النص الفعلي للوداع",
                "engagement_question": "السؤال المحدد للجمهور"
            }}
        }}
    }},
    "cultural_context": {{
        "proverbs_sayings": [
            "أمثال وحكم مرتبطة مباشرة بالموضوع المطروح",
            "مقولات مأثورة ذات صلة بالموضوع المحدد"
        ],
        "regional_references": [
            "مراجع محلية وإقليمية متعلقة بالموضوع",
            "تجارب عربية أو محلية مرتبطة بالموضوع"
        ],
        "shared_experiences": [
            "تجارب جماعية عربية مرتبطة بالموضوع",
            "ذكريات أو مواقف مشتركة متعلقة بالموضوع"
        ],
        "contemporary_relevance": [
            "أحداث جارية مرتبطة بالموضوع",
            "تطورات حديثة في المجال المطروح"
        ]
    }},
    "language_style": {{
        "formality_level": "مستوى الرسمية في اللغة",
        "dialect_touches": "لمسات لهجية خفيفة مناسبة",
        "vocabulary_richness": "ثراء المفردات المستخدمة",
        "sentence_variety": "تنويع في طول وتركيب الجمل"
    }},
    "technical_notes": {{
        "pacing_guidance": "إرشادات حول إيقاع الحديث",
        "pause_points": "نقاط توقف طبيعية",
        "emphasis_moments": "لحظات تأكيد وتشديد",
        "background_music_cues": "إشارات للموسيقى التصويرية"
    }}
}}

مهم جداً - متطلبات المحتوى:
- لا تكتب أوصافاً عامة مثل "ترحيب عام" أو "تقديم الموضوع"
- اكتب المحتوى الفعلي والنصوص المحددة التي سيقولها المقدم والضيف
- كل قيمة يجب أن تحتوي على نص حقيقي وليس وصفاً لما يجب فعله
- استخدم أسماء الشخصيات الفعلية (أحمد، نور) في النصوص
- اجعل كل محتوى مرتبطاً مباشرة بموضوع {topic}
- أرجع المخطط بصيغة JSON صحيحة فقط
- لا تضع علامات ```json في البداية أو ``` في النهاية
- لا تكتب أي نص إضافي قبل أو بعد JSON

مثال على ما هو مطلوب:
بدلاً من: "ترحيب عام بالمستمعين"
اكتب: "أهلاً وسهلاً بكم في حلقة جديدة من بودكاست التقنية والمستقبل، أنا أحمد المصري معكم اليوم"

بدلاً من: "تقديم الموضوع"
اكتب: "اليوم سنتحدث عن {topic}، هذا الموضوع المذهل الذي قد يغير حياتنا كلياً"
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في تصميم البودكاست العربية وتخصص في خلق محتوى طبيعي وتلقائي باللغة العربية الفصحى. الأسلوب المطلوب: {style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        
        return response.choices[0].message.content

    def generate_outline_with_style(self, topic, information, host_persona, guest_persona, style="حواري", duration="30 دقيقة"):
        """
        Generate outline with specific style considerations
        Styles: حواري (conversational), تعليمي (educational), ترفيهي (entertaining), تحليلي (analytical)
        """
        # Now this method just calls the main method which already handles style
        return self.generate_outline(topic, information, host_persona, guest_persona, style, duration)

    def validate_outline(self, outline_json):
        """
        Validate that the generated outline contains all required elements
        """
        required_keys = [
            "episode_topic", "personas", "conversation_flow", 
            "cultural_context", "language_style", "technical_notes"
        ]
        
        try:
            import json
            outline = json.loads(outline_json)
            
            missing_keys = []
            for key in required_keys:
                if key not in outline:
                    missing_keys.append(key)
            
            if missing_keys:
                return False, f"Missing required keys: {missing_keys}"
            
            return True, "Outline validation successful"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

In [4]:
# # ---------------------------------------------------------------
# #  New sample run: العنوسة في المجتمع العربي episode (entertaining style)
# # ---------------------------------------------------------------

# outline_generator = ImprovedPodcastOutlineGenerator(deployment, "gpt-4o")

# topic = "ظاهرة العنوسة في المجتمع العربي: أسباب وحلول"

# information = (
#    "ظاهرة العنوسة تشهد ارتفاعاً ملحوظاً في المجتمعات العربية، حيث تشير الإحصائيات إلى أن 30% من النساء في دول الخليج "
#    "و 25% من الرجال تجاوزوا سن الثلاثين دون زواج، مقارنة بـ 15% قبل عقدين. "
#    "الأسباب متعددة تشمل ارتفاع تكاليف الزواج والسكن، تغير الأولويات المهنية والتعليمية، صعوبة التوافق بين الشريكين، "
#    "وتأثير وسائل التواصل الاجتماعي على توقعات الشباب من الزواج. "
#    "العوامل الاقتصادية تلعب دوراً كبيراً حيث تصل تكلفة الزواج في بعض الدول العربية إلى 200 ألف دولار شاملة المهر والحفل والسكن. "
#    "التغيرات الاجتماعية مثل دخول المرأة سوق العمل بقوة، السفر للدراسة، وتقبل المجتمع للعزوبية أكثر من الماضي "
#    "أدت لتأخير قرارات الزواج. "
#    "الحلول المقترحة تتضمن برامج التيسير الحكومية، منصات التعارف المحترمة، ورش التأهيل للزواج، "
#    "وحملات توعية لتغيير النظرة المجتمعية حول تكاليف الزواج والعمر المناسب له."
# )

# host_persona = """فيصل العتيبي - العمر: 38
# المهنة/الخلفية: إعلامي ومقدم برامج اجتماعية من الرياض، متزوج وأب لثلاثة أطفال
# الشخصية: دبلوماسي، مرح، يجيد التعامل مع المواضيع الحساسة بخفة دم، يحب سرد القصص الشخصية
# OCEAN: انفتاح عالٍ، ضمير عالٍ، انبساط عالٍ جداً، وداعة عالية، عصابية منخفضة
# أسلوب التحدث: يستخدم الفكاهة لكسر الحرج، يشارك تجاربه الشخصية، يطرح أسئلة مباشرة لكن بلطف"""

# guest_persona = """د. نورا السالم - العمر: 42
# المهنة/الخلفية: استشارية علم اجتماع وخبيرة في قضايا المرأة والأسرة من بيروت، عزباء بالاختيار
# الشخصية: ذكية، صريحة، واثقة من نفسها، تملك حس دعابة عالي، لا تخجل من مناقشة المواضيع الشائكة
# OCEAN: انفتاح عالٍ جداً، ضمير عالٍ، انبساط متوسط، وداعة متوسطة، عصابية منخفضة
# أسلوب التحدث: تحليل علمي مع لمسة شخصية، تستخدم البيانات والإحصائيات، تتحدث بصراحة عن تجربتها"""

# # entertaining style
# outline_text = outline_generator.generate_outline(
#    topic,
#    information,
#    host_persona,
#    guest_persona,
#    style="ترفيهي"
# )

# print("Generated Outline:\n", outline_text)




# ---------------------------------------------------------------
#  New sample run: AI in Education episode (entertaining style)
# ---------------------------------------------------------------

outline_generator = ImprovedPodcastOutlineGenerator(deployment, "gpt-4o")

topic = "الذكاء الاصطناعي في التعليم: كيف يغيّر مستقبل التعلّم؟"

information = (
    "يتوقع أن يتجاوز حجم سوق تقنيات الذكاء الاصطناعي في التعليم عالمياً 30 مليار دولار بحلول 2030، "
    "بنمو سنوي يقارب 30٪. في العالم العربي، تبنت السعودية منصة «مدرستي» التي تستخدم التحليلات التعلّمية "
    "لتخصيص الدروس، فيما أطلقت الإمارات استراتيجية «مدرسة المستقبل» المبنية على مساعدات تدريس ذكية. "
    "التطبيقات تشمل أنظمة التعلّم التكيّفي، تصحيح الواجبات آلياً، وتوليد محتوى تعليمي فوري. "
    "الفوائد المحتملة: تحسين استيعاب الطلاب، تخفيف عبء المعلم الإداري، وتوفير تعليم شخصي لذوي الإعاقة. "
    "لكن التحديات تتضمن مخاوف الخصوصية، التحيز الخوارزمي، فجوة المهارات الرقمية بين المعلمين، "
    "والتكلفة الأولية العالية لتبنّي الحلول المتقدمة. دراسات اليونسكو 2024 تشير إلى أن 42٪ من المدارس "
    "في المنطقة لم تجهّز بعد بالبنية التحتية التقنية الكافية، ما يجعل تكافؤ الفرص أحد أبرز القضايا."
)

host_persona = """مريم أبو زهرة - العمر: 24
المهنة/الخلفية: طالبة صحافة وإذاعية ناشئة من القاهرة، تغطي قصص التعليم والتحول الرقمي
الشخصية: حماسية، فضولية، تحب طرح الأسئلة البسيطة التي تمس حياة الطلاب
OCEAN: انفتاح عالٍ، ضمير متوسط، انبساط عالٍ، وداعة عالية، عصابية متوسطة
أسلوب التحدث: لغة سهلة، قصص طلابية، تعليقات مرحة خفيفة"""

guest_persona = """د. ليلى الخالدي - العمر: 45
المهنة/الخلفية: أستاذة دراسات مستقبلية وتقنيات تعليم في جامعة قطر، باحثة في استخدام الذكاء الاصطناعي للتعلّم
الشخصية: هادئة، متفائلة، تركّز على الأثر المجتمعي للتقنية
OCEAN: انفتاح عالٍ جداً، ضمير عالٍ، انبساط متوسط، وداعة عالية، عصابية منخفضة
أسلوب التحدث: شروح مبسّطة، أمثلة واقعية، اقتباسات علمية، تربط التقنية بتنمية المهارات"""

# entertaining style
outline_text = outline_generator.generate_outline(
    topic,
    information,
    host_persona,
    guest_persona,
    style="ترفيهي"
)

print("Generated Outline:\n", outline_text)


Generated Outline:
 {
    "episode_topic": "الذكاء الاصطناعي في التعليم: كيف يغيّر مستقبل التعلّم؟",
    "personas": {
        "host": {
            "name": "مريم أبو زهرة",
            "background": "طالبة صحافة وإذاعية ناشئة من القاهرة، تغطي قصص التعليم والتحول الرقمي",
            "speaking_style": "لغة سهلة ومباشرة مع قصص طلابية وتعليقات مرحة خفيفة"
        },
        "guest": {
            "name": "د. ليلى الخالدي",
            "background": "أستاذة دراسات مستقبلية وتقنيات تعليم في جامعة قطر، باحثة في استخدام الذكاء الاصطناعي للتعلّم",
            "speaking_style": "شروح مبسّطة مع أمثلة واقعية واقتباسات علمية تربط التقنية بتنمية المهارات"
        }
    },
    "conversation_flow": {
        "intro1": {
            "opening_line": "مرحباً وأهلاً بكم في بودكاست \"تعلّم المستقبل\"! أنا مريم أبو زهرة، واليوم لدينا موضوع شيق جداً سيجعلكم تفكرون في المدرسة بشكل مختلف تماماً.",
            "podcast_introduction": "في هذا البودكاست، نناقش كل ما يخص التعليم والتكنولوجيا، وكيف يمكن أن تغيّر 

In [5]:
class MSADialogueEnhancer:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def enhance_section(self, section_name, content, criteria, complexity="بسيط", episode_topic="", style_example=None):
        
        complexity_guidance = {
            "بسيط": "استخدم لغة بسيطة ومفهومة للجمهور العام، تجنب المصطلحات المعقدة",
            "متوسط": "امزج بين البساطة والتعمق، استخدم مصطلحات تقنية مع الشرح",
            "معقد": "استخدم مصطلحات متخصصة ومفاهيم متقدمة مع شرح تفصيلي"
        }
        
        # Topic context to avoid confusion
        topic_context = f"""
تذكر أن موضوع الحلقة هو: {episode_topic}
يجب أن يكون كل المحتوى مرتبطاً بهذا الموضوع تحديداً وليس أي موضوع آخر.
"""
        
        if style_example:
            prompt = f"""
هذا مثال على أسلوب الحوار المطلوب:
{style_example}

{topic_context}

بناءً على القسم التالي من البودكاست، قم بتطويره وتحسينه حسب المعايير التالية: {criteria}

مستوى التعقيد المطلوب: {complexity}
{complexity_guidance.get(complexity, '')}

القسم: {section_name}
المحتوى: {content}

المطلوب:
- تأكد أن المحتوى يتعلق بموضوع: {episode_topic}
- اكتب نصاً تفاعلياً وجذاباً باللغة العربية الفصحى
- أضف حشو طبيعي متوسط مثل: يعني، طبعاً، بصراحة، اممم، اههه
- استخدم 70% فصحى و 30% لمسات خليجية خفيفة
- أدرج مراجع ثقافية عند الحاجة
- اجعل النص طبيعياً وتلقائياً
"""
        else:
            prompt = f"""
{topic_context}

بناءً على القسم التالي من البودكاست، قم بتطويره وتحسينه حسب المعايير التالية: {criteria}

مستوى التعقيد المطلوب: {complexity}
{complexity_guidance.get(complexity, '')}

القسم: {section_name}
المحتوى: {content}

المطلوب:
- تأكد أن المحتوى يتعلق بموضوع: {episode_topic} فقط
- اكتب نصاً تفاعلياً وجذاباً باللغة العربية الفصحى
- أضف حشو طبيعي متوسط مثل: يعني، طبعاً، بصراحة، اممم، اههه، واو
- استخدم 70% فصحى و 30% لمسات خليجية خفيفة (مثل: شلون، وش رايك، الله يعطيك العافية)
- أدرج مراجع ثقافية مرتبطة بالموضوع عند الحاجة
- اجعل النص طبيعياً وتلقائياً
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "أنت كاتب سيناريو بودكاست محترف متخصص في المحتوى العربي الطبيعي والتلقائي."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8
        )
        return response.choices[0].message.content

In [6]:
class MSASpontaneousDialogueGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment
    
    def generate_dialogue(self, section, section_content, host_persona, guest_persona, complexity="بسيط", style="تعليمي", episode_topic="", style_example=None):
        
        complexity_guidance = {
            "بسيط": "اجعل الشرح بسيطاً ومفهوماً للجمهور العام",
            "متوسط": "امزج بين البساطة والعمق، أضف تفاصيل تقنية مع الشرح",
            "معقد": "استخدم مصطلحات متخصصة ونقاش متعمق مع شرح تفصيلي"
        }
        
        style_guidance = {
            "حواري": "ركز على الحوار الطبيعي والتفاعل الشخصي، أضف لحظات تداخل وتفاعل عفوي",
            "تعليمي": "اهتم بتقديم المعلومات بطريقة منظمة ومفصلة، استخدم أمثلة توضيحية",
            "ترفيهي": "أضف عناصر مرحة وقصص شخصية طريفة، استخدم الفكاهة المناسبة",
            "تحليلي": "ركز على التحليل العميق والنقاش المتخصص، استخدم بيانات وإحصائيات"
        }
        
        # MSA fillers for natural conversation
        fillers_guide = """
استخدم حشو المحادثة الطبيعي بكثافة متوسطة:
- تفكير: اممم، اههه، يعني كيف أقول، خلاص
- تأكيد: طبعاً، تماماً، بالضبط، صحيح
- تردد: يعنييييي، يعني، اه ما أدري، شوف
- انفعال: واو، يا الله، ما شاء الله، الله يعطيك العافية
- ربط: بس، لكن، وبعدين، يا أخي، اسمع
- خليجي خفيف: شلون، وش رايك، زين، ماشي الحال
"""
        
        # Topic context to avoid confusion
        topic_context = f"""
تذكر أن موضوع الحلقة هو: {episode_topic}
يجب أن يكون كل الحوار مرتبطاً بهذا الموضوع تحديداً وليس أي موضوع آخر.
"""
        
        if style_example:
            prompt = f"""
هذا مثال على أسلوب الحوار المطلوب:
{style_example}

{topic_context}

حول القسم التالي إلى حوار تلقائي وطبيعي بين المقدم والضيف:

المقدم: {host_persona}
الضيف: {guest_persona}

القسم: {section}
المحتوى: {section_content}

الأسلوب المطلوب: {style}
{style_guidance.get(style, '')}

مستوى التعقيد: {complexity}
{complexity_guidance.get(complexity, '')}

{fillers_guide}

متطلبات الحوار:
- تأكد أن الحوار يدور حول موضوع: {episode_topic}
- استخدم أسماء الشخصيات الفعلية في الحوار
- اجعل كل شخصية تتحدث بأسلوبها المميز
- أضف تفاعلات طبيعية: <happy>, <surprise>, <overlap> (فقط عند الحاجة)
- استخدم 70% فصحى و 30% لمسات خليجية
- أدرج مراجع ثقافية مرتبطة بالموضوع
- اجعل الحوار يتكيف مع الموضوع
- أضف فواصل طبيعية [pause: 2s] عند الحاجة
- ابدأ كل جملة بـ المقدم: أو الضيف:
"""
        else:
            prompt = f"""
{topic_context}

حول القسم التالي إلى حوار تلقائي وطبيعي بين المقدم والضيف باللغة العربية الفصحى:

المقدم: {host_persona}
الضيف: {guest_persona}

القسم: {section}
المحتوى: {section_content}

الأسلوب المطلوب: {style}
{style_guidance.get(style, '')}

مستوى التعقيد: {complexity}
{complexity_guidance.get(complexity, '')}

{fillers_guide}

متطلبات الحوار:
- تأكد أن الحوار يدور حول موضوع: {episode_topic} فقط
- استخدم أسماء الشخصيات الفعلية (أحمد، نور) في الحوار
- اجعل كل شخصية تتحدث بأسلوبها المميز حسب خلفيتها
- أضف تفاعلات طبيعية: <happy>, <surprise>, <overlap> (فقط عند الضرورة)
- استخدم 70% فصحى و 30% لمسات خليجية خفيفة
- أدرج مراجع ثقافية مرتبطة بالموضوع عند الحاجة
- اجعل الحوار يتكيف مع الموضوع
- أضف فواصل طبيعية [pause: 2s] عند الحاجة للتأمل
- ابدأ كل جملة بـ المقدم: أو الضيف:
- اجعل الحوار متماشياً مع الأسلوب المطلوب مع الحفاظ على التلقائية
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "أنت خبير في كتابة الحوارات التلقائية والطبيعية للبودكاست العربي. تخصصك في الأسلوب التعليمي الممتع."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8
        )
        return response.choices[0].message.content

In [7]:
def generate_msa_podcast_script(outline_text, host_persona, guest_persona, complexity="بسيط", style="تعليمي", deployment=None):
    """
    Generate complete MSA podcast script from outline
    
    Args:
        outline_text: JSON string of the podcast outline
        host_persona: Host persona description
        guest_persona: Guest persona description  
        complexity: بسيط/متوسط/معقد (simple/medium/complex)
        style: حواري/تعليمي/ترفيهي/تحليلي (conversational/educational/entertaining/analytical)
        deployment: OpenAI client deployment
    
    Returns:
        List of dictionaries with section dialogues
    """
    
    dialogue_generator = MSASpontaneousDialogueGenerator(deployment, "gpt-4o")
    enhancer = MSADialogueEnhancer(deployment, "gpt-4o")
    
    outline = json.loads(outline_text)
    
    # Extract topic from outline for context
    episode_topic = outline.get("episode_topic", "الموضوع المطروح")
    
    # Complete Arabic dialogue style examples
    arabic_dialogue_styles = {
        "حواري": {
            "host_example": "أحمد: يا أهلاً نور! كيف الحال؟ اممم... قوليلي، شو اللي خلاكِ تدخلي هذا المجال؟ <happy>",
            "guest_example": "نور: أهلاً أحمد! الله يعطيك العافية... يعني بصراحة، هاي قصة طويلة شوي [pause: 2s] بس باختصار، كنت أشوف المشاكل حولي وأقول: ليش ما نحلها بالتقنية؟"
        },
        "تعليمي": {
            "host_example": "أحمد: اممم... نور، ممكن تشرحي لنا بطريقة بسيطة، يعني شلون تشتغل هاي التقنية؟ <happy>",
            "guest_example": "نور: طبعاً أحمد! يعني... اههه كيف أشرح [pause: 2s] تخيل إنك عندك نظام ذكي جداً، بس هذا النظام مش إنسان، هو كمبيوتر! واو صح؟"
        },
        "ترفيهي": {
            "host_example": "أحمد: هاي نور! <happy> قوليلي، إيش أغرب موقف صار معكِ في الشغل؟ يعني شي يضحك؟",
            "guest_example": "نور: ههههه واو أحمد! بصراحة مواقف كثيرة... اممم مرة كنت أجرب البرنامج وفجأة [pause: 2s] خلاص ما عاد يشتغل! قعدت أصرخ: وين راح كودي؟! <surprise>"
        },
        "تحليلي": {
            "host_example": "أحمد: نور، بناءً على الإحصائيات الحديثة، وش رايكِ في التحديات الرئيسية اللي تواجه هذا المجال؟",
            "guest_example": "نور: سؤال ممتاز أحمد... يعني إذا نتكلم بشكل تحليلي، عندنا ثلاث تحديات أساسية [pause: 2s] أولها التقنية، ثانيها التمويل، وثالثها... اممم التقبل المجتمعي"
        }
    }
    
    # Get style examples based on the selected style
    style_examples = arabic_dialogue_styles.get(style, arabic_dialogue_styles['تعليمي'])
    podcast_dialogues = []
    
    # Process conversation flow sections
    conversation_flow = outline.get("conversation_flow", {})
    
    # Intro1
    if "intro1" in conversation_flow:
        intro1_content = conversation_flow["intro1"]
        content_text = " ".join([
            intro1_content.get("opening_line", ""),
            intro1_content.get("podcast_introduction", ""),
            intro1_content.get("episode_hook", "")
        ])
        
        enhanced_intro1 = enhancer.enhance_section(
            "المقدمة الأولى",
            content_text,
            criteria="جذاب، حماسي، ترحيبي، ومتحمس",
            complexity=complexity,
            episode_topic=episode_topic,
            style_example=style_examples["host_example"]
        )
        podcast_dialogues.append({"section": "intro1", "dialogue": enhanced_intro1})
    
    # Intro2  
    if "intro2" in conversation_flow:
        intro2_content = conversation_flow["intro2"]
        content_text = " ".join([
            intro2_content.get("topic_introduction", ""),
            intro2_content.get("guest_welcome", ""),
            intro2_content.get("guest_bio_highlight", "")
        ])
        
        enhanced_intro2 = dialogue_generator.generate_dialogue(
            "تقديم الموضوع والضيف",
            content_text,
            host_persona,
            guest_persona,
            complexity=complexity,
            style=style,
            episode_topic=episode_topic,
            style_example=f"{style_examples['host_example']} {style_examples['guest_example']}"
        )
        podcast_dialogues.append({"section": "intro2", "dialogue": enhanced_intro2})
    
    # Main Discussion Points
    if "main_discussion" in conversation_flow:
        main_points = conversation_flow["main_discussion"]
        
        for i, point in enumerate(main_points):
            point_title = point.get("point_title", f"النقطة {i+1}")
            
            # Build content from all point elements
            content_parts = [
                point.get("personal_angle", ""),
                " ".join(point.get("spontaneous_triggers", [])),
                point.get("disagreement_points", ""),
                " ".join(point.get("cultural_references", [])),
                point.get("natural_transitions", ""),
                point.get("emotional_triggers", "")
            ]
            section_content = " ".join([part for part in content_parts if part])
            
            spontaneous_dialogue = dialogue_generator.generate_dialogue(
                point_title,
                section_content,
                host_persona,
                guest_persona,
                complexity=complexity,
                style=style,
                episode_topic=episode_topic,
                style_example=f"{style_examples['host_example']} {style_examples['guest_example']}"
            )
            
            podcast_dialogues.append({
                "section": f"main_point_{i+1}",
                "title": point_title,
                "dialogue": spontaneous_dialogue
            })
    
    # Closing sections
    closing = conversation_flow.get("closing", {})
    
    # Conclusion
    if "conclusion" in closing:
        conclusion_content = closing["conclusion"]
        content_text = " ".join([
            conclusion_content.get("main_takeaways", ""),
            conclusion_content.get("guest_final_message", ""),
            conclusion_content.get("host_closing_thoughts", "")
        ])
        
        enhanced_conclusion = dialogue_generator.generate_dialogue(
            "الخلاصة",
            content_text,
            host_persona,
            guest_persona,
            complexity=complexity,
            style=style,
            episode_topic=episode_topic,
            style_example=f"{style_examples['host_example']} {style_examples['guest_example']}"
        )
        podcast_dialogues.append({"section": "conclusion", "dialogue": enhanced_conclusion})
    
    # Outro
    if "outro" in closing:
        outro_content = closing["outro"]
        content_text = " ".join([
            outro_content.get("guest_appreciation", ""),
            outro_content.get("audience_thanks", ""),
            outro_content.get("call_to_action", ""),
            outro_content.get("next_episode_teaser", "")
        ])
        
        enhanced_outro = enhancer.enhance_section(
            "الختام",
            content_text,
            criteria="دافئ، ممتن، ملهم، ومشوق للحلقة القادمة",
            complexity=complexity,
            episode_topic=episode_topic,
            style_example=style_examples["host_example"]
        )
        podcast_dialogues.append({"section": "outro", "dialogue": enhanced_outro})
    
    return podcast_dialogues

In [8]:
# Usage example:

# Initialize
dialogue_generator = MSASpontaneousDialogueGenerator(deployment, "gpt-4o")
enhancer = MSADialogueEnhancer(deployment, "gpt-4o")

# Generate script with complexity level
podcast_script = generate_msa_podcast_script(
    outline_text,
    host_persona,
    guest_persona, 
    complexity="متوسط",  # بسيط، متوسط، معقد
    deployment=deployment
)

# Print results
for section in podcast_script:
    print(f"\\n=== {section['section']} ===")
    print(section['dialogue'])


\n=== intro1 ===
### المقدمة الأولى (مُعاد تطويرها)

مريم: أهلاً وسهلاً في بودكاست "تعلّم المستقبل"، يا هلا ويا مرحبا بكل مستمعينا الأعزاء! معكم مريم أبو زهرة، واليوم عندنا موضوع... أقدر أقول إنه من النوع اللي يخلّي الواحد يعيد التفكير في كل شيء نعرفه عن المدرسة والتعليم. يعني، هل تخيلتوا يوم من الأيام أن ذكاء اصطناعي ممكن يغيّر الطريقة اللي نتعلّم فيها؟ مو بس تغييرات بسيطة، لا، تغييرات جذرية!

مريم: بصراحة، كثير منّا لما يسمع كلمة "ذكاء اصطناعي"، أول شي يخطر في باله هو روبوتات ذكية أو أفلام الخيال العلمي، صح؟ لكن... اممم خلونا نكون واقعيين شوي، هذا المجال ما صار مجرد فكرة من الأفلام، صار جزء من حياتنا اليومية. واليوم، نتكلم عن كيف هالتقنية قاعدة تلعب دور كبير في التعليم، يعني في كيف الطلاب يتعلمون وكيف المعلمين يدرّسون. 

مريم: طبعاً، الذكاء الاصطناعي مو مجرد روبوتات تصحح الواجبات أو تُساعد الطلاب على فهم الدروس، الموضوع أعمق بكثير. بنسأل أسئلة مثل: هل ممكن الذكاء الاصطناعي يفهم احتياجات الطالب بشكل فردي؟ هل يقدر يساعد في تقليل الضغط على المعلمين؟ و... يمكن الأهم، هل هو فعلاً مفيد؟ أم